In [9]:
!pip install matplotlib seaborn pandas
!pip install xgboost
!pip install catboost

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


  Using cached catboost-1.2.5-cp39-cp39-win_amd64.whl (101.2 MB)


You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [36]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score

In [63]:
# 데이터 불러오기
df = load_data(file_path)

# CD 열의 결측치를 0으로 대치하는 함수 정의
def fill_missing_cd_with_zero(df):
    """
    CD 열의 결측치를 0으로 대치하는 함수

    Parameters:
    - df (pd.DataFrame): 데이터프레임

    Returns:
    - df (pd.DataFrame): 결측치가 대치된 데이터프레임
    """
    df['CD'] = df['CD'].fillna(0)
    return df

# 제거할 열 목록
columns_to_drop = ['ID','YEAR', 'SAMPLE_TRANSFER_DAY', 'COMPONENT_ARBITRARY', 'U100', 'U75', 'U50', 'U25', 'U14', 'U6', 'U4','FH2O','FNOX','FOPTIMETHGLY','FOXID','FSO4','FTBN','FUEL','SOOTPERCENTAGE']

# 열 제거
df = df.drop(columns=columns_to_drop)

# CD 열의 결측치를 0으로 대치
df = fill_missing_cd_with_zero(df)

# 결과 출력
print(df.head())
print(df.info())

   ANONYMOUS_1  ANONYMOUS_2  AG   AL    B  BA  BE    CA   CD  CO  ...  SB  \
0         1486          200   0    3   93   0   0  3059  0.0   0  ...   0   
1         1350          375   0    2   19   0   0  2978  0.0   0  ...   0   
2         2415          200   0  110    1   1   0    17  0.0   0  ...   0   
3         7389          200   0    8    3   0   0  1960  0.0   0  ...   2   
4         3954          200   0    1  157   0   0    71  0.0   0  ...   0   

    SI  SN  TI  U20  V  V100    V40   ZN  Y_LABEL  
0  427   2   5  NaN  0   NaN  154.0   75        0  
1    0   0   0  4.0  0   NaN   44.0  652        0  
2    0   1   0  3.0  0   NaN   72.6  412        1  
3    1   1   0  NaN  0   NaN  133.3    7        0  
4    2   0   0  NaN  0   NaN  133.1  128        0  

[5 rows x 35 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14095 entries, 0 to 14094
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ANON

In [64]:
# K 변수의 결측치를 가지는 행만 선택
k_missing = df[df['K'].isnull()]

# 결측치를 가지지 않는 K 변수와 관련 변수들 선택
k_not_missing = df.dropna(subset=['K'])

# 피처와 타겟 변수 설정
X = k_not_missing[['NA', 'SI']]
y = k_not_missing['K']

# 모델 학습
model = LinearRegression()
model.fit(X, y)

# 결측치를 가지는 데이터에 대한 예측
X_missing = k_missing[['NA', 'SI']]
k_missing['K'] = model.predict(X_missing)

# 결측치를 포함한 데이터프레임을 업데이트
df.update(k_missing)

print(df['K'].isnull().sum())  # 결측치 확인

0


C:\Users\USER\AppData\Local\Temp\ipykernel_21280\2603757911.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k_missing['K'] = model.predict(X_missing)


In [66]:
X = df.drop(columns=['Y_LABEL'])
y = df['Y_LABEL']

# Robust Scaler 적용
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)


In [67]:
# 테스트 데이터 분할
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [68]:
# catboost
# CatBoost 분류 모델 생성 및 훈련
cat = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0)
cat.fit(X_tr, y_tr)
# 하이퍼파라미터 그리드 정의
param_grid = {
    'iterations': [100, 200, 500],
    'depth': [4, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'l2_leaf_reg': [1, 3, 5],
    'border_count': [32, 50, 100]
}



In [69]:
# GridSearchCV 초기화
grid_search = GridSearchCV(estimator=cat, param_grid=param_grid, cv=3, scoring='f1', verbose=1, n_jobs=-1)

# GridSearchCV를 사용한 모델 훈련
grid_search.fit(X_tr, y_tr)
# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)



Fitting 3 folds for each of 243 candidates, totalling 729 fits
Best Hyperparameters: {'border_count': 32, 'depth': 4, 'iterations': 100, 'l2_leaf_reg': 3, 'learning_rate': 0.2}


In [70]:
# 최적의 하이퍼파라미터로 모델 생성
best_cat = grid_search.best_estimator_

# 테스트 세트에 대해 예측
y_pred = best_cat.predict(X_val)

# F1 스코어 계산
f1 = f1_score(y_val, y_pred, average='macro')
print("F1 Score: {:.2f}".format(f1))

F1 Score: 0.83


코드 통합본

In [11]:
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

# 데이터 로드
df = pd.read_csv('C:\\Users\\USER\\projects\\bae\\실습자료\\train(component4).csv')

# 제거할 열 목록
columns_to_drop = ['ID','YEAR', 'SAMPLE_TRANSFER_DAY', 'COMPONENT_ARBITRARY', 'U100', 'U75', 'U50', 'U25', 'U14', 'U6', 'U4','FH2O','FNOX','FOPTIMETHGLY','FOXID','FSO4','FTBN','FUEL','SOOTPERCENTAGE']

# 열 제거
df = df.drop(columns=columns_to_drop)

# CD 열의 결측치를 0으로 대치
df['CD'] =df['CD'].fillna(df['CD'].mode()[0])

# K 변수의 결측치를 가지는 행만 선택
k_missing = df[df['K'].isnull()]

# 결측치를 가지지 않는 K 변수와 관련 변수들 선택
k_not_missing = df.dropna(subset=['K'])

# 피처와 타겟 변수 설정
X = k_not_missing[['NA', 'SI']]
y = k_not_missing['K']

# 모델 학습
model = LinearRegression()
model.fit(X, y)

# 결측치를 가지는 데이터에 대한 예측
X_missing = k_missing[['NA', 'SI']]
k_missing['K'] = model.predict(X_missing)

# 결측치를 포함한 데이터프레임을 업데이트
df.update(k_missing)

X = df.drop(columns=['Y_LABEL'])
y = df['Y_LABEL']

# Robust Scaler 적용
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# 테스트 데이터 분할
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# catboost
# CatBoost 분류 모델 생성 및 훈련
cat = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0)
cat.fit(X_tr, y_tr)
# 하이퍼파라미터 그리드 정의
param_grid = {
    'iterations': [100, 200, 500],
    'depth': [4, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'l2_leaf_reg': [1, 3, 5],
    'border_count': [32, 50, 100]
}

# GridSearchCV 초기화
grid_search = GridSearchCV(estimator=cat, param_grid=param_grid, cv=3, scoring='f1', verbose=1, n_jobs=-1)

# GridSearchCV를 사용한 모델 훈련
grid_search.fit(X_tr, y_tr)
# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

# 최적의 하이퍼파라미터로 모델 생성
best_cat = grid_search.best_estimator_

# 테스트 세트에 대해 예측
y_pred = best_cat.predict(X_val)

# F1 스코어 계산
f1 = f1_score(y_val, y_pred, average='macro')
print("F1 Score: {:.2f}".format(f1))

C:\Users\USER\AppData\Local\Temp\ipykernel_4228\57203193.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k_missing['K'] = model.predict(X_missing)


Fitting 3 folds for each of 243 candidates, totalling 729 fits
Best Hyperparameters: {'border_count': 32, 'depth': 4, 'iterations': 100, 'l2_leaf_reg': 3, 'learning_rate': 0.2}
F1 Score: 0.83
